In [77]:
import cv2
import numpy as np
import PIL
import json
import scipy.interpolate as si

In [78]:
img = cv2.imread("./dog.jpg")
f = open("./dog.json")
data = json.load(f)
annotation = data["annotations"][0]["shape"]

In [80]:
def get_tck_param(annoation):
    for key, values in annotation.items():
            if key =="t":
                t = values
            elif key == "c":
                c = values
            elif key == "k":
                k = values
    return t,c,k

In [81]:
def calculate_spine_params(c, n=100, degree=k):
    ci = np.asarray(c)
    count = ci.shape[0]
    degree = np.clip(degree,1,count-1)
    # Knot vector
    knot_v = np.array([0]*degree + list(range(count-degree+1)) + [count-degree]*degree,dtype='int')
    # Query range
    query_r = np.linspace(0,(count-degree),n)
    return np.array(si.splev(query_r, (knot_v,ci.T,degree))).T

In [82]:
def image_spline(c):
    points = np.array(c, np.int32)
    for pt in points:
        cv2.circle(img,(pt[0],pt[1]), 5, (0,255,0), -1)
    points = points.reshape((-1, 1, 2))
    color = (255, 0, 0)
    thickness = 1
    cv2.polylines(img, [points],False, color, thickness,lineType = cv2.LINE_AA)
    p = calculate_spine_params(points,n=100)[:,0,:]    
    color = (0, 0, 255)
    thickness = 2
    cv2.polylines(img, [p.astype(np.int32)],False, color, thickness,  lineType = cv2.LINE_AA)
    return img

In [83]:
t,c,k = get_tck_param(annotation)
spine_param = calculate_spine_params(c, n=100, degree=k)
img = image_spline(c)
cv2.imshow("Result spline image",img)    
cv2.waitKey(0)  
cv2.destroyAllWindows()